In [ ]:
!pip install --user py4j
!pip install --user pyspark

In [5]:

import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
import os
import pandas as pd

os.environ['HTTP_PROXY'] = "http://172.17.0.1:3080"


In [3]:

spark = SparkSession \
    .builder \
    .appName("dsx-test") \
    .config("spark.driver.allowMultipleContexts",  True) \
    .getOrCreate()


In [10]:

customers = pd.read_csv('../datasets/customer_details_noquotes.csv', dtype='object')
#customers.info()

from pyspark.sql.types import *
schema = StructType([
    StructField("name", StringType(), True),
    StructField("subscription_date", StringType(), True),
    StructField("phone_brand", StringType(), True),
    StructField("phone_model", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("state", StringType(), True),
    StructField("account_length", StringType(), True),
    StructField("international_plan", StringType(), True),
    StructField("voice_mail_plan", StringType(), True),
    StructField("number_vmail_messages", StringType(), True),
    StructField("customer_service_calls", StringType(), True),
    StructField("churn", StringType(), True)])

customers = spark.createDataFrame(customers, schema)


In [11]:

customers_apple = customers.filter(customers.phone_brand == 'Apple')
customers_apple.show(3)


+---------------+-------------------+-----------+-------------+--------------+------+--------------+------------------+---------------+---------------------+----------------------+-----+
|           name|  subscription_date|phone_brand|  phone_model|         phone| state|account_length|international_plan|voice_mail_plan|number_vmail_messages|customer_service_calls|churn|
+---------------+-------------------+-----------+-------------+--------------+------+--------------+------------------+---------------+---------------------+----------------------+-----+
|   Grant Rachel|01/04/2017 00:00:00|      Apple|     iPhone 6|(415) 382-4657|Kansas|           128|             false|           true|                   25|                     1|false|
| Vaughan Audrey|14/07/2016 00:00:00|      Apple|iPhone 6 Plus|(408) 350-8884| Idaho|            85|             false|           true|                   27|                     1|false|
|Parsons Cameron|01/09/2015 00:00:00|      Apple| iPhone 6Plus|(5

In [12]:

from pyspark.sql.functions import countDistinct

by_states = customers.groupBy('state').agg(countDistinct('phone'))
by_states.show()


+------------+---------------------+
|       state|count(DISTINCT phone)|
+------------+---------------------+
|        Utah|                   72|
|      Hawaii|                   53|
|   Minnesota|                   84|
|        Ohio|                   78|
|          DC|                   54|
|      Oregon|                   78|
|    Arkansas|                   54|
|       Texas|                   72|
|North Dakota|                   62|
|Pennsylvania|                   45|
| Connecticut|                   74|
|    Nebraska|                   60|
|     Vermont|                   72|
|      Nevada|                   66|
|  Washington|                   67|
|    Illinois|                   58|
|    Oklahoma|                   61|
|    Delaware|                   60|
|      Alaska|                   51|
|  New Mexico|                   62|
+------------+---------------------+
only showing top 20 rows



In [15]:

from hdfs import InsecureClient

client_hdfs = InsecureClient('http://192.168.54.10:50070', user='bruno.bonnin')

filename = '/user/bruno.bonnin/by_states.csv'
#filename = '/user/cloudera/customer_details.csv'
with client_hdfs.write(filename, encoding = 'utf-8', overwrite = True) as writer:
    by_states.toPandas().to_csv(writer)
    